<a href="https://colab.research.google.com/github/Cesare-Caputo/PyPSA-gridtracer/blob/main/create_maxar_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dependancies

In [ ]:
# ! pip uninstall opencv-python -y
# ! pip install opencv-python

In [ ]:
!pip install geopandas rtree
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.chdir('/content/drive/My Drive/PyPSA_Africa_images/')
import importlib
import sys
sys.path.append(os.getcwd()+'/detect-energy/src/make_data/')
import geopandas as gpd
import pandas as pd
import fiftyone as fo
import json

## Tile Tifs

In [ ]:
def fix_annots(file):
    '''
    adds information on the 'iscrowd' property
    for training with detectron2 to an annotation json file
    made by fiftyone
    '''
    dictionary = json.load(open(file))
    for annot in dictionary['annotations']:
        annot['iscrowd'] = 0

    with open(file, "w") as f:
        json.dump(dictionary, f)


def create_maxar_dataset(country, base, out_dir, img_dir, gdf_file, max_img=1000):

    print('Assuming image with 512 and heigth 512.')
    width = 512
    height = 512

    os.mkdir(out_dir)

    df = gpd.read_file(gdf_file)
    print(df.head())

    # set up datasets for current country
    try:
        dataset = fo.Dataset(name=country+'_val')
    except:
        dataset = fo.load_dataset(country+'_val')
        dataset.delete()
        dataset = fo.Dataset(name=country+'_val')
        # _dataset2 = fo.load_dataset("my_second_dataset")
    dataset.persistent = False

    label_field = 'ground_truth'

    for i, row in df.iterrows():

        if i == max_img:
            break

        example_name = row.filename

        sample = fo.Sample(filepath=os.path.join(img_dir, example_name))
        bbox = [
                row.ul_x / width,
                row.ul_y / height,
                (row.lr_x - row.ul_x) / width,
                (row.lr_y - row.ul_y) / height,
                ]
        detections = list()
        detections.append(fo.Detection(label='tower', bounding_box=bbox))

        sample['ground_truth'] = fo.Detections(detections=detections)
        dataset.add_sample(sample)

    dataset.export(
        export_dir=out_dir,
        dataset_type=fo.types.COCODetectionDataset,
        label_field=label_field,
       )


# country = 'drc'
# tower_gdf_file = 'CD_tower_examples.geojson'
# base = '/content/drive/MyDrive/PyPSA_Africa_images/'

# out_dir = os.path.join(base, 'datasets', country+'_val/')
# img_dir = os.path.join(base, 'maxar', country, 'images_512')
# gdf_file = os.path.join(img_dir, tower_gdf_file)

# create_maxar_dataset(country, base, out_dir, img_dir, gdf_file, max_img=30)
# fix_annots(os.path.join(out_dir, 'labels.json'))

country = 'bangladesh'
tower_gdf_file = 'BD_tower_examples.geojson'
base = '/content/drive/MyDrive/PyPSA_Africa_images/'

out_dir = os.path.join(base, 'datasets', country+'_val/')
img_dir = os.path.join(base, 'maxar', country, 'images_512')
gdf_file = os.path.join(img_dir, tower_gdf_file)

create_maxar_dataset(country, base, out_dir, img_dir, gdf_file, max_img=300000)
fix_annots(os.path.join(out_dir, 'labels.json'))



Assuming image with 512 and heigth 512.
             filename   ul_x   ul_y   lr_x   lr_y  \
0  BD_88569650140.png  106.0  336.0  166.0  386.0   
1  BD_16111043050.png   47.0  123.0  107.0  173.0   
2  BD_16110834300.png   99.0  345.0  159.0  395.0   
3  BD_49663381980.png  460.0   39.0  520.0   89.0   
4  BD_49663380940.png  308.0  152.0  368.0  202.0   

                                            geometry  
0  POLYGON ((90.36388 23.73156, 90.36630 23.73156...  
1  POLYGON ((90.49231 23.76211, 90.49474 23.76211...  
2  POLYGON ((90.40373 23.67306, 90.40615 23.67306...  
3  POLYGON ((90.35303 23.85258, 90.35546 23.85258...  
4  POLYGON ((90.43036 23.88132, 90.43278 23.88132...  
Directory '/content/drive/MyDrive/PyPSA_Africa_images/datasets/bangladesh_val/' already exists; export will be merged with existing files


 100% |███████████████| 3939/3939 [11.2m elapsed, 0s remaining, 6.6 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 3939/3939 [11.2m elapsed, 0s remaining, 6.6 samples/s]      


In [ ]:
dataset = fo.Dataset.from_dir(
            dataset_type=fo.types.COCODetectionDataset,
            data_path=out_dir+'data/',
            labels_path=out_dir+'labels.json',)

session = fo.launch_app(dataset)

 100% |███████████████████| 30/30 [91.3ms elapsed, 0s remaining, 328.5 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 30/30 [91.3ms elapsed, 0s remaining, 328.5 samples/s]     


In [ ]:
print(os.getcwd())
print(sys.path)
import vis
# import main
# import create_coco_format
#import filter_tiles
#import make_examples

In [ ]:
os.getcwd()

In [ ]:
!pwd

In [ ]:
%cd ..

In [ ]:
%cd /content/drive/MyDrive/PyPSA_Africa_images/MAXAR_256

In [ ]:
from PIL import Image
from osgeo import osr, gdal
import numpy as np
from gdalconst import *
from shapely.geometry import Polygon
import geopandas as gpd
import folium
import os
import rtree
import matplotlib.pyplot as plt
# import pygeos

# os.chdir(os.path.dirname(os.path.abspath(__file__)))  # move up to parent directory

#%%
def make_polygon_list(path, resume_file=None):
    """
    Creates GeoDataFrame, checks the tif files in a desired dir
    and adds their geometries (with respective file name) as row
    to GeoDataFrame
    -----------
    Arguments:
    path : (str)
        directory with tif files
    resume_file : (str or None)
        if None: initializes net GeoDataFrame
        otherwise loads (and resumes from) existing one
    """

    tif_files = []

    # iterate over all .tif files in desired dir
    for filename in os.listdir(path):
        if filename.endswith(".tif"):
            tif_files.append(os.path.join(path, filename))

    # add data to existing dataframe if desired
    try:
        coverage = gpd.read_file(resume_file)
        print(f"Adding coverage to {resume_file}")
    except:
        coverage = gpd.GeoDataFrame({"filename": [], "geometry": []})

    # extract geometries of .tif files and add to geodataframe
    for file in tif_files:
        try:
            info = gdal.Info(file, format="json")
        except SystemError:
            print(f'Skipping raster {file}')
            continue
        corners = info["cornerCoordinates"]
        poly = Polygon((
                    corners["upperLeft"],
                    corners["upperRight"],
                    corners["lowerRight"],
                    corners["lowerLeft"]
                        ))
        coverage = coverage.append({"filename": file, "geometry": poly}, ignore_index=True)

    coverage = coverage.set_crs(epsg=4326)

    return coverage

#%%
def make_examples(assets,
                  coverage,
                  img_path="/../examples/",
                  max_length=10,
                  height=512,
                  width=512,
                  bbox_height=25,
                  bbox_width=30,
                  random_offset = True,
                  seed = None,
                  examples_per_tower=1):
    """
    Expects dataframe of energy infrastructure assets in fn. Iterates over df
    and for each asset finds the respective polygon from coverage geodataframe.
    Then opens the .tif file that corresponds to the polygon and cuts out the
    respective pixels. The pixels are turned into a png files and saves.
    Additionally, the respective file name, geometry, bbox, is written into
    a new GeoDataFrame
    ----------
    Arguments:
    assets : (str or GeoDataFrame)
        path to or GeoDataFrame itself of energy assets
    coverage : (str or GeoDataFrame)
        path to or GeoDataFrame itself of geometries satellite imagery
    img_path : (str)
        path to directory where resulting examples will vbe stored
    max_length : (int / None)
        restricts number of images created if desired
    height : (int)
        number of pixels in y direction
    width : (int)
        number of pixels in x direction
    random_offset : (bool)
        set false for zero offset otherwise randomly set
    seed : (int)
        seed value for numpy random generator
    examples_per_tower : (int)
        number of examples generated per tower
    ----------
    Returns:
    dataset : (GeoDataFrame)
        df of created examples. contains for each image:
            filename (ending in .png)
            for bbox: upperleft and lowerright
            geometry as Polygon
    Also
    Saves created examples as .png files to img_path
    In the same directory the GeoJSON of the respective
    GeoDataFrame dataset is stored in the same directory
    """

    # img_path = os.path.abspath("") + img_path

    if isinstance(assets, str): assets = gpd.read_file(assets)
    if isinstance(coverage, str): coverage = gpd.read_file(coverage)

    # set up resulting dataset of examples (with towers)
    dataset = gpd.GeoDataFrame({"filename": [],
                                "ul_x": [], "ul_y": [], "lr_x": [], "lr_y": [],
                                "geometry": []}).set_crs(epsg=4326)

    # bounding_box = gpd.GeoDataFrame({"filename": [], "geometry": []})

    assets = gpd.sjoin(assets, coverage, how="inner").set_crs(epsg=4326)
    assets = assets.drop(["index_right"], axis=1)

    # for image labeling
    pos = [i for i, sign in enumerate(img_path) if sign is '/'][-1]
    prefix = img_path[pos+1:]

    print(f"Maxiumum Number of Examples: {len(assets)}")
    if len(assets)<max_length:
        max_length = len(assets)

    if seed is not None:
        np.random.seed(seed)

    # iterate over .tif files
    for i, image in enumerate(coverage["filename"]):

        # open files
        ds = gdal.Open(image)
        info = gdal.Info(image, format="json")
        bands = [ds.GetRasterBand(i) for i in range(1, 4)]

        # extract relevant geographical data
        transform = info["geoTransform"]
        upper_left = np.array([transform[0], transform[3]])
        pixel_size = np.array([transform[1], transform[5]])

        # iterate over assets in that image
        for row in assets[assets["filename"] == image].itertuples():

            # (default=1) generates multiple examples for every examples
            for j in range(examples_per_tower):

                # compute relevant pixels
                p = row.geometry
                coords = np.array([p.xy[0][0], p.xy[1][0]])
                pixels = np.around((coords - upper_left) / pixel_size)
                pixels -= np.array([width // 2, height // 2])
                # add random offset (8 is minimal distance to image boundary)

                offset = np.zeros(2)
                if random_offset is True:
                    offset[0] = np.random.randint(-width//2 + 20, width//2 - 20)
                    offset[1] = np.random.randint(-height//2 + 20, height//2 - 20)

                pixels -= offset
                x, y = int(pixels[0]), int(pixels[1])

                # set up image and new filename
                filename = str(int(10*row.id+j))
                new_img = np.zeros((height, width, 3), dtype=np.uint8)

                # transfer pixel data
                try:
                    for i in range(3):
                        new_img[:,:,i] = bands[i].ReadAsArray(x, y, width, height)
                except:
                    continue

                # create Polygon of created image
                img_corner = upper_left + pixels*pixel_size
                img_polygon = Polygon([
                                       img_corner,
                                       img_corner + pixel_size*np.array([width,0]),
                                       img_corner + pixel_size*np.array([width,height]),
                                       img_corner + pixel_size*np.array([0,height])
                                      ])

                # get pixels of bbox; format: (ul_x, ul_y, lr_x, lr_y)
                ul = np.array([width//2, height//2]) + offset
                bbox = [
                        ul[0] - bbox_width // 2,
                        ul[1] - bbox_height // 2,
                        ul[0] + bbox_width // 2,
                        ul[1] + bbox_height // 2
                ]

                # _, ax = plt.subplots(1, 1, figsize=(8, 8))
                # ax.imshow(new_img)
                # ax.scatter([bbox[0], bbox[2]], [bbox[1], bbox[3]], s=100, c='r')
                # plt.show()

                # do not include images that are contain blank spots
                black_threshold = 10
                binarr = np.where(new_img < black_threshold, 1, 0) #black_point is upper limit
                # Find Total sum of 2D array thresh
                total = sum(map(sum, binarr))
                ratio = total/height/width

                if (ratio > 0.5).sum() == 3:
                    print('filtered by black borders!')
                    continue

                # do not include images that contain clouds
                white_point = 180
                # Put threshold to make it binary
                binarr = np.where(new_img>white_point, 1, 0) #white point is lower limit
                # Find Total sum of 2D array thresh
                total = sum(map(sum, binarr))
                ratio = total/height/width
                if (ratio > 0.45).sum() == 3:
                    print('filtered by cloudy!')
                    continue

                # exclude images that are blurry
                blurr_threshold = 0.65
                # _, s, _ = np.linalg.svd(new_img)
                _, s, _ = np.linalg.svd(new_img.sum(axis=2))
                sv_num = new_img.shape[0] // 50
                ratio = s[:sv_num].sum() / s.sum()
                if ratio > blurr_threshold and not 'australia' in image:
                    print('filtered by blurry!')
                    continue

                # transform array to image
                img = Image.fromarray(new_img, 'RGB')
                img.save(img_path + filename + ".png", quality=100)

                # geo_bbox = gpd.GeoSeries(p).set_crs("EPSG:4326").to_crs("EPSG:3857").buffer(30)
                # geo_bbox = geo_bbox.to_crs("EPSG:4326")
                # geo_bbox = gpd.GeoSeries(p)
                # bounding_box = bounding_box.append({"filename": filename,"geometry": geo_bbox[0]},ignore_index=True)
                # bounding_box.set_crs("EPSG:4326", inplace=True)

                # add resulting image to dataset
                dataset = dataset.append({"filename": prefix + filename + '.png',
                                          "ul_x": bbox[0],
                                          "ul_y": bbox[1],
                                          "lr_x": bbox[2],
                                          "lr_y": bbox[3],
                                          "geometry": img_polygon},
                                          ignore_index=True)

                # # save results inbetween
                # if len(dataset)%50 == 0:
                #     dataset = dataset.set_crs(epsg=4326)
                #     dataset.to_file(os.path.join(img_path, "tower_examples.geojson"), driver="GeoJSON")
                #     bounding_box.to_file(img_path + "tower_bbox.geojson", driver="GeoJSON")
                if len(dataset)%50 == 0:
                    print("Created {} Examples!".format(len(dataset)))


                # early stoppage
                if len(dataset) == max_length:
                    prev_len = len(dataset)
                    dataset.drop_duplicates(subset=['filename'], inplace=True) # For assets that exitst in overlapping coverage areas
                    new_len = len(dataset)
                    if new_len < prev_len:
                        print(f"removed {prev_len - new_len} duplicates")
                        print(f"remaining {new_len} examples")
                    dataset.to_file(img_path + "tower_examples.geojson", driver="GeoJSON")
                    # bounding_box.to_file(img_path + "tower_bbox.geojson", driver="GeoJSON")
                    return None

    dataset.to_file(img_path + "tower_examples.geojson", driver="GeoJSON")



In [ ]:
dirs = [
        # 'australia',
        'drc',
        'malawi',
        'sierra_leone',
        'ghana',
        'bangladesh',
        ]
country_dict = {'sierra_leone': 'SL', 'ghana':'GH', 'malawi': 'MW',
                    'drc': 'CD', 'australia': 'AU', 'bangladesh': 'BD'}
base_path = os.path.join('/content', 'drive', 'My Drive', 'PyPSA_Africa_images')

for country in dirs:
    country_code = country_dict[country]

    out_path = os.path.join(base_path, 'maxar', country, 'images_512')
    # out_path = os.path.join(base_path, 'maxar', 'img_dump')

    max_ratio = 1.
    print(f"Making Examples for {country}!")

    # assess satellite imagery

    data_path = os.path.join(base_path, 'maxar', country, 'raw')

    coverage_polygon = make_polygon_list(data_path)
    towers_file = os.path.join(data_path, f"{country_code}_raw_towers.geojson")

    tower_df = gpd.read_file(towers_file)
    num_towers = len(tower_df)
    max_towers = int(max_ratio*num_towers)

    # creates the examples
    make_examples(towers_file,
                  coverage_polygon,
                  img_path=os.path.join(out_path, f"{country_code}_"),
                  max_length=max_towers,
                  height=512,
                  width=512,
                  bbox_height=50,
                  bbox_width=60,
                  random_offset=True,
                  seed=2021,
                  )


Making Examples for drc!
Maxiumum Number of Examples: 12517
filtered by blurry!
filtered by blurry!
filtered by black borders!
filtered by blurry!
filtered by black borders!
filtered by black borders!
Created 50 Examples!
Created 100 Examples!
filtered by black borders!
filtered by black borders!
Created 150 Examples!
Created 200 Examples!
Created 250 Examples!
Created 300 Examples!
Created 350 Examples!
Created 400 Examples!
filtered by blurry!
Created 450 Examples!
Created 500 Examples!
Created 550 Examples!
Created 600 Examples!
Created 650 Examples!
Created 700 Examples!
filtered by blurry!
filtered by black borders!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
Created 750 Examples!
filtered by blurry!
filtered by blurry!
filtered by cloudy!
filtered by cloudy!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered by blurry!
filtered 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import numpy as np

def vis_image(examples, path, show_box=True, idx=0):
    """
    Opens image of desired index by reading the filename from examples
    Shows the image, (with bounding box if desired)
    ----------
    Arguments:
    examples : (GeoDataFrame or str)
        gdf of all examples - used to obtain filename and bbox
    path : (str)
        to directory of images and dataframe
    show_box : (bool)
        bounding box is shown if True
    idx : (int)
        desired example

    ----------
    Returns:
    -
    """

    if isinstance(examples, str): examples = gpd.read_file(path + examples)

    example = examples.iloc[idx]
    plt.figure(figsize=(10, 7))
    img = plt.imread(path + example.filename)
    fig = plt.imshow(img)

    if show_box:
        bbox = [example.ul_x, example.ul_y, example.lr_x, example.lr_y]
        rect = plt.Rectangle(xy=(bbox[0], bbox[1]), width=bbox[2] - bbox[0],
                             height=bbox[3] - bbox[1], fill=False,
                             edgecolor="r", linewidth=2)
        fig.axes.add_patch(rect)

    plt.show()

In [ ]:

country = "MW"
gjson_path = country + '_tower_examples.geojson'

path = os.path.join(base_path, 'MAXAR_256/')
examples = gpd.read_file(os.path.join(path, gjson_path))


print(examples.tail())
print(len(examples))
for _ in range(60):
    num_ex = np.random.randint(0, 300)
    path = os.path.join(base_path, 'MAXAR_256/')
    vis_image(gjson_path, path, idx=num_ex)

In [ ]:
merged_df = gpd.GeoDataFrame()
example_path = os.path.join(base_path, 'MAXAR_256')
file = os.path.join(example_path, f"{country_code}_tower_examples.geojson")

for country in dirs:
    country_code = country_dict[country]
    tower_examples_df = gpd.read_file(file)
    merged_df = pd.concat([merged_df, tower_examples_df])

merged_df = merged_df.drop_duplicates(subset='filename')
merged_df.to_file(os.path.join(example_path, "tower_examples.geojson"), driver="GeoJSON")
display(merged_df)

In [ ]:
df = gpd.read_file(os.path.join(os.getcwd(), 'MAXAR_256', 'tower_examples.geojson'))
files = os.listdir(os.getcwd() + '/MAXAR_256')

df['exists'] = df['filename'].apply(lambda entry: entry in files)

print(df['exists'].sum())



In [ ]:
file = os.path.join(base_path, 'MAXAR_256', 'tower_examples.geojson')
filter_tiles.filter_images(file, delete_filtered=True, black_point=50,
                           dark_threshold=.5, white_point=150, cloudy_threshold=.45,
                           blurry_threshold=.65)

In [ ]:
filter_tiles.verify_df_img('tower_examples_clean.geojson')

## Train-Test Splits

In [ ]:
def split_data(gdf_path, train_ratio=0.8, seed=202):
    df = gpd.read_file(gdf_path)
    df_dir = os.path.dirname(os.path.abspath(gdf_path)) +'/'
    train = df.sample(frac=train_ratio, random_state=seed) #random state is a seed value

    val = df.drop(train.index)
    val = val.sample(frac=1, random_state=seed)
    train.to_file(df_dir + "tower_examples_train.geojson", driver="GeoJSON")
    val.to_file(df_dir + "tower_examples_val.geojson", driver="GeoJSON")

    return train, val

train,val = split_data(os.path.join(base_path, 'MAXAR_256', "tower_examples_clean.geojson"),
                                    train_ratio=0.8, seed=202)
display(train)
display(val)

## Create COCO

In [ ]:
train_file = os.path.join(base_path, 'MAXAR_256', 'tower_examples_train.geojson')
val_file = os.path.join(base_path, 'MAXAR_256', 'tower_examples_val.geojson')

create_coco_format.to_coco(train_file, '_train', height=256, width=256)
create_coco_format.to_coco(val_file, '_val', height=256, width=256)

In [ ]:
!pip install fiftyone

In [ ]:
import fiftyone as fo

In [ ]:
# The directory containing the source images
data_path = "/content/drive/MyDrive/PyPSA_Africa_images/MAXAR_256/"
# The path to the COCO labels JSON file
labels_path = "/content/drive/MyDrive/PyPSA_Africa_images/MAXAR_256/tower_coco_val.json"

In [ ]:
# Import the dataset
import os

os.chdir('/content/drive/MyDrive/PyPSA_Africa_images/MAXAR_256/')

# The directory containing the source images
mode = 'train'
data_path = "/content/drive/MyDrive/PyPSA_Africa_images/datasets/maxar_"+mode
# The path to the COCO labels JSON file
labels_path = data_path + '/labels.json'

dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path,
)

In [ ]:
session = fo.launch_app(dataset)

In [ ]:
import os
os.path.join('test', 'file.txt')

## transfer to dataset directory

In [ ]:
from shutil import copyfile

settypes = ['train', 'val']

for settype in settypes:
    origin = os.path.join(base_path, "MAXAR_256")
    destpath = os.path.join(base_path, 'datasets', 'maxar_'+settype)

    print(os.listdir(destpath))
    for file in [os.path.join(destpath, f) for f in os.listdir(destpath)]:
        os.remove(file)

    gdf =  os.path.join(origin, 'tower_examples_'+settype+'.geojson')
    gdf = gpd.read_file(gdf)

    for i, row in gdf.iterrows():

        file = row['filename']
        copyfile(os.path.join(origin, file), os.path.join(destpath, file))

    copyfile(os.path.join(origin, 'tower_coco_'+settype+'.json'),
             os.path.join(destpath, 'labels.json'))





